In [1]:
import topmost
from topmost.data import download_dataset

device = "cuda" # or "cpu"
dataset_dir = "./datasets/20NG"
# download_dataset('20NG', cache_path='./datasets')

d:\Anaconda\envs\rl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import torch.version
torch.__version__

'2.4.0+cu124'

In [3]:
########################### Neural Topic Models ####################################
# dataset for neural topic models.
# For combinedTM, add contextual_embed=True.
dataset = topmost.data.BasicDataset(dataset_dir, read_labels=True, device=device)

train_size:  11314
test_size:  7532
vocab_size:  5000
average length: 110.543


In [4]:
# for batch in dataset.train_dataloader:
#     print(batch.shape)

In [5]:
# create a model
# model = topmost.ProdLDA(dataset.vocab_size)
model = topmost.ETM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
# model = topmost.DecTM(dataset.vocab_size)
# model = topmost.TSCTM(dataset.vocab_size)
# model = topmost.CombinedTM(dataset.vocab_size, dataset.contextual_embed_size)
# model = topmost.NSTM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
# model = topmost.ECRTM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
model = model.to(device)

# create a trainer
trainer = topmost.BasicTrainer(model, dataset, verbose=True)

# train the model
top_words, train_theta = trainer.train()

100%|██████████| 200/200 [01:17<00:00,  2.59it/s]


Topic 0: max get difference places point really food nice got little welcome mean red noticed anything
Topic 1: gun people right police law local guns weapons control get crime kill anti court one
Topic 2: god jesus bible believe christian christ church life christians religion faith people say many one
Topic 3: subject key interested writes keith eric marc deal alan larry rsa strip literature charles bobby
Topic 4: lines center research institute motif based original pittsburgh ray base chicago reference mellon society carnegie
Topic 5: window send line lines message display image please screen see advance color look section right
Topic 6: like find look real rather stuff make large black little images white sort means cover
Topic 7: chip bill robert president house clinton bob package said points chips letter asked asking henry
Topic 8: use problem using need help used anyone code work problems ibm build host uses application
Topic 9: mail usa card fax newsreader tin bbs cards modem 

In [6]:
from utils import save_model, load_model

save_model(model)
model = load_model(model)

Model weights saved at saved_model\model_weights.pth


In [7]:
# import torch
# from topmost import Preprocess

# preprocess = Preprocess()

# new_docs = [
#     "This is a new document about space, including words like space, satellite, launch, orbit.",
#     "This is a new document about Microsoft Windows, including words like windows, files, dos."
# ]

# parsed_new_docs, new_bow = preprocess.parse(new_docs, vocab=dataset.vocab)

# # Chuyển đổi sparse matrix sang dense NumPy array
# new_bow_dense = new_bow.toarray()  # Hoặc .todense() cũng được

# # Chuyển sang PyTorch tensor
# new_theta = trainer.test(torch.tensor(new_bow_dense, device=device).float())

# print(new_theta.argmax(1))


In [8]:
# from transformers import BertTokenizer, BertModel

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertModel.from_pretrained("bert-base-uncased")

In [9]:
# # Câu cần chuyển thành embedding
# sentence = "This is a sentence for embedding using BERT."

# # Tokenize và tạo tensor đầu vào
# inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)

# # Lấy embedding từ BERT
# with torch.no_grad():
#     outputs = model(**inputs)

# # Lấy embedding của toàn bộ câu (dùng CLS token hoặc mean pooling)
# cls_embedding = outputs.last_hidden_state[:, 0, :]  # Lấy embedding của [CLS] token
# mean_embedding = outputs.last_hidden_state.mean(dim=1)  # Trung bình tất cả token

# # In kích thước embedding
# print("CLS embedding shape:", cls_embedding.shape)  # (1, 768)
# print("Mean embedding shape:", mean_embedding.shape)  # (1, 768)


In [10]:
from environment import Environment
env = Environment(dataset = dataset, topic_model = model, device = "cuda")

In [11]:
### testing environment
def get_state_shape(state):
    print(state.keys())
    for key in state.keys():
        print(state[key].shape)
state = env.reset()
get_state_shape(state)

parsing texts: 100%|██████████| 1/1 [00:00<00:00, 999.60it/s]
d:\Anaconda\envs\rl\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


dict_keys(['doc_embed', 'mean_topic_embeds'])
torch.Size([1, 768])
torch.Size([1, 768])


In [12]:
env.get_k_top_words("This is a new document about Microsoft Windows, including words like windows, files, dos.", testing = True)

parsing texts: 100%|██████████| 1/1 [00:00<00:00, 661.46it/s]


['file',
 'windows',
 'files',
 'graphics',
 'run',
 'output',
 'source',
 'server',
 'free',
 'programs',
 'printer',
 'unix',
 'error',
 'user',
 'color',
 'quality',
 'size',
 'multi',
 'multiple',
 'directory']

In [19]:
done = False
while done == False:
    state, reward, done = env.step(action = "windows")
if done:
    env.print_current_state()
    state = env.reset()

get_state_shape(state)
reward, done

### ENVIRONMENT STATE ###
Current time step: 20
Document:  david subject real reason politicians want guns organization software support consulting lines politicians want eliminate private ownership guns general public starts tax increases needed fund federal government ever higher percentage tax revenue goes pay interest national currently every tax dollar collected dave beginning look forward reaching taxes pay interest national point federal government business lack funds
K-Top words:  ['government', 'public', 'national', 'state', 'law', 'american', 'private', 'states', 'organization', 'federal', 'health', 'administration', 'rights', 'care', 'congress', 'americans', 'policy', 'united', 'free', 'work']


parsing texts: 100%|██████████| 1/1 [00:00<00:00, 62.83it/s]


dict_keys(['doc_embed', 'mean_topic_embeds'])
torch.Size([1, 768])
torch.Size([1, 768])


(tensor([-15.3566]), True)